Inicialmente importamos las librerías para trabajar con PySpark

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from datetime import date

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.bronze;
CREATE VOLUME IF NOT EXISTS workspace.default.test_volume;
SHOW VOLUMES IN workspace.default;

Leemos los archivos desde el volumen "test_volume" y creamos los dataframes para luegoi crear las tablas en el esquema bronze

In [0]:
path_files = "/Volumes/workspace/default/test_volume/"

# Schema para empleados
empleados_schema = StructType([
    StructField("id_vendedor", IntegerType(), True),
    StructField("sucursal", StringType(), True),
    StructField("nombre", StringType(), True)
])

df_empleados = (
        spark.read.format("csv")
                  .option("header", "true")
                  .schema(empleados_schema)
                  .load(path_files + "empleados.csv")
)                  
df_empleados.show()

In [0]:
# Schema para fact
fact_schema = StructType([
    StructField("fecha_ventas", StringType(), True),
    StructField("sku", IntegerType(), True),
    StructField("id_vendedor", IntegerType(), True),
    StructField("cantidad", IntegerType(), True)
])

df_fact = (
        spark.read.format("csv")
                  .option("sep", ";")
                  .option("header", "true")
                  .schema(fact_schema)
                  .load(path_files + "fact.csv")
)

# Convertimos de string a DATE
df_fact = df_fact.withColumn(
    "fecha_ventas", F.to_date(F.col("fecha_ventas"), "d/M/yyyy")
)
df_fact.show()

In [0]:
# Schema para locales
locales_schema = StructType([
    StructField("id_sucursal", IntegerType(), True),
    StructField("nombre", StringType(), True),
    StructField("tipo", StringType(), True)
])

df_locales = (
        spark.read.format("csv")
                  .option("sep", ";")
                  .option("header", "true")
                  .schema(locales_schema)
                  .load(path_files + "locales.csv")
)                  
df_locales.show()

In [0]:
# Schema para producto
producto_schema = StructType([
    StructField("id_producto", IntegerType(), True),
    StructField("familia", StringType(), True),
    StructField("nombre", StringType(), True),
    StructField("precio_unitario", DoubleType(), True)
])

df_producto = (
        spark.read.format("csv")
                  .option("sep", ";")
                  .option("header", "true")
                  .schema(producto_schema)
                  .load(path_files + "producto.csv")
)                  
df_producto.show()

Creamos las tablas temporales

In [0]:
df_empleados.createOrReplaceTempView("empleados")
df_fact.createOrReplaceTempView("fact")
df_locales.createOrReplaceTempView("locales")
df_producto.createOrReplaceTempView("producto")

Crear tablas en el Metasktore (schema and bronze)

In [0]:
bronze_table_empleados = "bronze.raw_empleados"

spark.sql(f"DROP TABLE IF EXISTS {bronze_table_empleados}")
spark.sql(f"CREATE TABLE {bronze_table_empleados} AS SELECT * FROM empleados")

spark.sql(f"SELECT * FROM {bronze_table_empleados}").show()

In [0]:
bronze_table_locales = "bronze.raw_locales"

spark.sql(f"DROP TABLE IF EXISTS {bronze_table_locales}")
spark.sql(f"CREATE TABLE {bronze_table_locales} AS SELECT * FROM locales")

spark.sql(f"SELECT * FROM {bronze_table_locales}").show()

In [0]:
bronze_table_fact = "bronze.raw_fact"

spark.sql(f"DROP TABLE IF EXISTS {bronze_table_fact}")
spark.sql(f"CREATE TABLE {bronze_table_fact} AS SELECT * FROM fact")

spark.sql(f"SELECT * FROM {bronze_table_fact}").show()

In [0]:
bronze_table_productos  = "bronze.raw_productos"

spark.sql(f"DROP TABLE IF EXISTS {bronze_table_productos }")
spark.sql(f"CREATE TABLE {bronze_table_productos } AS SELECT * FROM producto")

spark.sql(f"SELECT * FROM {bronze_table_productos }").show()

2. Validar con una consulta que se hayan cargado correctamente todos los registros de cada uno de los archivos.

In [0]:
# Empleados
table = spark.table("bronze.raw_empleados")

# Diferencias en ambos sentidos
diff = df_empleados.subtract(table).union(table.subtract(df_empleados))

# Validación
assert diff.count() == 0, "❌ Hay diferencias entre el DataFrame y la tabla"
print("✅ Ambos tienen los mismos registros")


In [0]:
# Locales
table = spark.table("bronze.raw_locales ")

# Diferencias en ambos sentidos
diff = df_locales.subtract(table).union(table.subtract(df_locales))

# Validación
assert diff.count() == 0, "❌ Hay diferencias entre el DataFrame y la tabla"
print("✅ Ambos tienen los mismos registros")

In [0]:
# fact 
table = spark.table("bronze.raw_fact ")

# Diferencias en ambos sentidos
diff = df_fact.subtract(table).union(table.subtract(df_fact))

# Validación
assert diff.count() == 0, "❌ Hay diferencias entre el DataFrame y la tabla"
print("✅ Ambos tienen los mismos registros")

In [0]:
# productos 
table = spark.table("bronze.raw_productos ")

# Diferencias en ambos sentidos
diff = df_producto.subtract(table).union(table.subtract(df_producto))

# Validación
assert diff.count() == 0, "❌ Hay diferencias entre el DataFrame y la tabla"
print("✅ Ambos tienen los mismos registros")